# 텐서보드

https://www.tensorflow.org/tensorboard/get_started

2.0이 안 깔려 있으면 텐서보드를 쓰지 못한다.

In [1]:
# !pip install watermark

워터마크는 주피터 노트북에 버전같은거 찍어주는 애다.

### water mark

In [2]:
%load_ext watermark

In [3]:
%lsmagic

Available line magics:
%alias  %alias_magic  %autoawait  %autocall  %automagic  %autosave  %bookmark  %cd  %clear  %cls  %colors  %conda  %config  %connect_info  %copy  %ddir  %debug  %dhist  %dirs  %doctest_mode  %echo  %ed  %edit  %env  %gui  %hist  %history  %killbgscripts  %ldir  %less  %load  %load_ext  %loadpy  %logoff  %logon  %logstart  %logstate  %logstop  %ls  %lsmagic  %macro  %magic  %matplotlib  %mkdir  %more  %notebook  %page  %pastebin  %pdb  %pdef  %pdoc  %pfile  %pinfo  %pinfo2  %pip  %popd  %pprint  %precision  %prun  %psearch  %psource  %pushd  %pwd  %pycat  %pylab  %qtconsole  %quickref  %recall  %rehashx  %reload_ext  %ren  %rep  %rerun  %reset  %reset_selective  %rmdir  %run  %save  %sc  %set_env  %store  %sx  %system  %tb  %time  %timeit  %unalias  %unload_ext  %watermark  %who  %who_ls  %whos  %xdel  %xmode

Available cell magics:
%%!  %%HTML  %%SVG  %%bash  %%capture  %%cmd  %%debug  %%file  %%html  %%javascript  %%js  %%latex  %%markdown  %%perl  %%prun  %%pyp

In [4]:
%watermark -chan -d -pnumpy,sklearn,scipy,tensorflow
#콤마뒤에 공백이 있으면 안된다.
# 얘전에 내가 어떤 버전으로 시작했는지 알 수 있다.
# 컨퍼런스에서 자주 이런식으로 사용한다. 근데 안 예뻐서 선생님은 안씀.

han 2019-12-06 

numpy 1.17.3
sklearn 0.0
scipy 1.3.1
tensorflow 2.0.0


### versoin-information

In [5]:
# !pip install version-information

In [6]:
%load_ext version_information

In [7]:
%version_information -numpy, scipy, sklearn, tensorflow, tensorboard

Software versions
Python 3.7.5 64bit [MSC v.1916 64 bit (AMD64)]
IPython 7.9.0
OS Windows 10 10.0.17763 SP0
-numpy Invalid requirement, parse error at "'-numpy'"
scipy 1.3.1
sklearn 0.21.3
tensorflow 2.0.0
tensorboard 2.0.0
Fri Dec 06 17:11:06 2019 ¢¥eCN©öI¡¾©ö C¡ÍA¨ª¨öA

### tensorboard

In [8]:
%load_ext tensorboard

### datetime

In [9]:
import datetime
import tensorflow as tf

In [10]:
# !clear -rf ./logs/ # 리룩스에서만 되나보다.

In [11]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

def create_model():
    return tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
  ])

train_on_batch에서 학습이 가능하다. 이럴 때 콜백개념이 조금 달라서 쓰기가 복잡하다. 처음부터 만드는 경우도 쓰기가 복잡하다. 

fit 쓰는 이유 중 하나가, 텐서보드를 쉽게 볼 수 있기 때문이다.

In [12]:
model = create_model()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [13]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = 'logs_01')

In [14]:
model.fit(x=x_train, 
          y=y_train, 
          epochs=1, 
          validation_data=(x_test, y_test), 
          callbacks=[tensorboard_callback])

Train on 60000 samples, validate on 10000 samples
60000/60000 [==============================] - 5s 80us/sample - loss: 0.2178 - accuracy: 0.9360 - val_loss: 0.1074 - val_accuracy: 0.9665


In [15]:
# !dir # 윈도우에서는 이렇게 연다.

In [16]:
# %tensorboard --logdir logs_01

# Displaying image data in TensorBoard

In [17]:
from datetime import datetime
import io
import itertools
from packaging import version
from six.moves import range

import tensorflow as tf
from tensorflow import keras

import matplotlib.pyplot as plt
import numpy as np
import sklearn.metrics

print("TensorFlow version: ", tf.__version__)
assert version.parse(tf.__version__).release[0] >= 2, \
    "This notebook requires TensorFlow 2.0 or above."

TensorFlow version:  2.0.0


# Download the Fashion-MNIST dataset

In [18]:
# Download the data. The data is already divided into train and test.
# The labels are integers representing classes.
fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = \
    fashion_mnist.load_data()

# Names of the integer classes, i.e., 0 -> T-short/top, 1 -> Trouser, etc.
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 
    'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

### Visualizing a single image

In [19]:
print("Shape: ", train_images[0].shape)
print("Label: ", train_labels[0], "->", class_names[train_labels[0]])

Shape:  (28, 28)
Label:  9 -> Ankle boot


In [20]:
img = np.reshape(train_images[0], (-1, 28, 28, 1))

In [21]:
tf.summary.image("Training data", img) # 텐서였다. 직접적으로 못본다.

<tf.Tensor: id=7533, shape=(), dtype=bool, numpy=False>

In [22]:
file_writer = tf.summary.create_file_writer('logs_01')

In [23]:
with file_writer.as_default(): # as_default는 1.0버전의 유산이다.
    tf.summary.image("Training data", img, step=0)

In [24]:
dir(file_writer.as_default()) # __enter__, __exit__가 튀어나온다.
# 이걸 했을 때는 with 구문쓰라는 것이다.

['__abstractmethods__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_recreate_cm',
 'args',
 'func',
 'gen',
 'kwds']

### 이름 맞추기 꼼수

In [25]:
import datetime

In [26]:
dir(datetime)

['MAXYEAR',
 'MINYEAR',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'date',
 'datetime',
 'datetime_CAPI',
 'sys',
 'time',
 'timedelta',
 'timezone',
 'tzinfo']

In [27]:
x = datetime.date(2019, 12, 29)
y = datetime.date(2019, 12, 27)

In [28]:
x-y # 이런식으로 시간연산이 가능하다.

datetime.timedelta(days=2)

In [29]:
z = datetime.timedelta(3)
x + z # timedelta는 이런식으로 쓴다.

datetime.date(2020, 1, 1)

In [30]:
datetime.datetime.now() # 사람이 읽기 좋게 time.time을 바꿔줌

datetime.datetime(2019, 12, 6, 17, 11, 14, 921)

In [31]:
import time
time.time() # 현재 밀리세컨드

1575619874.0069366

In [32]:
datetime.datetime.strftime(x, format = '%y-%m-%d') # 포맷에 맞춰 바꿔줌

'19-12-29'

In [33]:
datetime.datetime.strptime('19-12-29', '%y-%m-%d') # 포맷에 맞추어 객체로 만들어줌

datetime.datetime(2019, 12, 29, 0, 0)

### 이미지 보기 총정리

In [34]:
logdir = "logs/train_data/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# Creates a file writer for the log directory.
file_writer = tf.summary.create_file_writer(logdir)
# ★★★ 이거 실행할때마다 로그파일 하나씩 생긴다.

# Using the file writer, log the reshaped image.
with file_writer.as_default(): # file writer가 최근에 만든 녀석을 불러온다.
    tf.summary.image("Training data", img, 
                     step=16) # step은 개조레벨이다.

In [35]:
# !tensorboard --logdir "logs/train_data/" # 안된다.

나는 느낌표 텐서보드가 안된다...

AUTOTUNE: 레벨과 밝기를 알아서 맞춰준다.

이미지 처리는, 밝기와 명암에 따라서 성능차이가 심하게 난다. 그래서 이게 텐서보드가 있다. 맷플롯핍보다 더 편하게 해결할 수 있어서, 이것을 쓴다.

 - reload_ext
 - unload_ext
 - load_ext
 
이 세가지는 좀 알아둬라